In [1]:
import configparser
cfg = configparser.ConfigParser()
cfg.optionxform = str
cfg.read('/home/sarth/rootdir/assets/global.ini')
cfg = {s: dict(cfg.items(s)) for s in cfg.sections()}
PATHS = cfg['PATHS']

In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe
import networkx as nx

import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt

from shapely.geometry import Point, Polygon, MultiPolygon
from shapely.ops import unary_union

import glob
import os
import itertools
import tqdm
import gc
import time
import pickle

from joblib import Parallel, delayed

In [6]:
DIRNAME = '30min_CWatM_India'
basin_name = 'mahanadi'
SAVE_PATH = os.path.join(PATHS['devp_datasets'], DIRNAME)
os.makedirs(SAVE_PATH, exist_ok=True)
resolution = 0.50

In [7]:
files = sorted(glob.glob(os.path.join(PATHS['IndiaWRIS'], 'basins', basin_name, '*', 'shapefile.shp')))
subbasins = gpd.GeoDataFrame(pd.concat([gpd.read_file(f) for f in files], ignore_index=True))
subbasins = subbasins.to_crs(epsg=4326)
subbasins['gauge_id'] = [os.path.basename(os.path.dirname(f)) for f in files]
subbasins = subbasins.drop(columns=['basin'])
subbasins

,geometry,gauge_id
0,"POLYGON ((81.27792 22.50542, 81.27958 22.50542...",andhiyar_khore
1,"POLYGON ((82.49042 23.55958, 82.49292 23.55958...",bamnidhi
2,"POLYGON ((81.90125 20.91708, 81.90292 20.91708...",baronda
3,"POLYGON ((82.49042 23.55958, 82.49292 23.55958...",basantpur
4,"POLYGON ((81.96792 22.77958, 81.97292 22.77958...",ghatora
5,"POLYGON ((81.96792 22.77958, 81.97292 22.77958...",jondhra
6,"POLYGON ((82.57292 20.74375, 82.57375 20.74375...",kantamal
7,"POLYGON ((82.57292 20.74375, 82.57375 20.74375...",kesinga
8,"POLYGON ((80.82542 21.31708, 80.82625 21.31708...",kotni
9,"POLYGON ((83.39708 23.07125, 83.39875 23.07125...",kurubhata


In [8]:
# Load Attributes
indiawris_basin = pd.read_csv(os.path.join(PATHS['IndiaWRIS'], 'basins', basin_name, 'indiawris_basin.csv'), index_col=0)
indiawris_basin

,River Point Name,Latitude,Longitude,NaNs,Days with Observations,Zero of Gauge,MERIT_Longitude,MERIT_Latitude,Gauge_MERIT_Distance,uparea (in m2),uparea (in km2)
0,andhiyar_khore,21.833889,81.597500,5.30,14184,252.0,81.597500,21.834167,30.887,2.166368e+09,2166.368402
1,bamnidhi,21.908486,82.713640,4.45,14311,223.0,82.710000,21.910000,411.522,9.811223e+09,9811.223484
2,baronda,20.910000,81.888056,6.64,13983,283.0,81.888333,20.910000,28.853,3.193768e+09,3193.768107
3,basantpur,21.738500,82.785942,4.64,14282,206.0,82.778333,21.722500,1944.963,5.898489e+10,58984.885999
4,ghatora,22.048592,82.221956,5.35,14176,246.0,82.222500,22.047500,133.739,3.037212e+09,3037.211720
5,jondhra,21.712492,82.333106,5.66,14130,219.0,82.332500,21.715000,285.855,2.979901e+10,29799.010658
6,kantamal,20.658333,83.732069,5.28,14186,118.0,83.730000,20.660833,351.621,2.026583e+10,20265.829359
7,kesinga,20.285831,83.221333,5.58,14142,166.0,83.249167,20.262500,3893.397,1.197787e+10,11977.870300
8,kotni,21.240000,81.250278,4.57,14293,268.0,81.250833,21.236667,375.095,7.036693e+09,7036.693190
9,kurubhata,21.981278,83.210600,12.18,13153,215.0,83.211667,21.980833,120.579,4.752530e+09,4752.530476


In [9]:
uparea = xr.open_dataset(os.path.join(PATHS['gis_ldd'], 'CWatM_30min/upstream_area_km2.nc'))
ds_varname = list(uparea.data_vars)[0]
uparea = uparea[ds_varname]
uparea.load()

<xarray.DataArray 'upstream_area_km2' (lat: 360, lon: 720)> Size: 1MB
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
  * lat      (lat) float64 3kB 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
  * lon      (lon) float64 6kB -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
Attributes:
    transform:      [   0.5    0.  -180.     0.    -0.5   90. ]
    crs:            +proj=longlat +datum=WGS84 +no_defs=True
    res:            [0.5 0.5]
    is_tiled:       1
    nodatavals:     -3.4028230607370965e+38
    scales:         1.0
    offsets:        0.0
    DataType:       Generic
    AREA_OR_POINT:  Area

In [10]:
ldd = xr.open_dataset(os.path.join(PATHS['gis_ldd'], 'CWatM_30min', 'ldd.nc'))
ds_var_name = list(ldd.data_vars)[0]
ldd = ldd[ds_var_name]
ldd.load()

<xarray.DataArray 'ldd' (lat: 360, lon: 720)> Size: 1MB
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
  * lon      (lon) float64 6kB -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
  * lat      (lat) float64 3kB 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
Attributes:
    standard_name:   ldd
    long_name:       drainage_network
    units:           -
    esri_pe_string:  GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1...

In [11]:
def get_contributing_region(ldd, lon_snapped, lat_snapped):
    # ldd_loc = lambda x,y: int(ldd.sel(lat = y, lon = x, method = 'nearest').values)
    def ldd_loc(x, y):
        value = ldd.sel(lon=x, lat=y, method = 'nearest').values
        if np.isnan(value):
            return 5
        else:
            return int(value)
    # print(lon_snapped, lat_snapped, ldd_loc(lon_snapped, lat_snapped))

    def get_nbr_list(x, y, res = resolution):
        # get the 8 neighbors
        nbr = [
            (x-res, y-res), (x, y-res), (x+res, y-res), 
            (x-res, y), (x+res, y),
            (x-res, y+res), (x, y+res), (x+res, y+res)]
        # Round to 3
        nbr = [(round(x, 3), round(y, 3)) for x, y in nbr]
        return nbr
    
    def get_nbr_ldd(nbr):
        return [ldd_loc(x, y) for x, y in nbr]
    
    def get_nbr_contributing(nbr, ldd_nbr):
        # Sequence: 9, 8, 7, 6, 4, 3, 2, 1
        seq_contributing = [9, 8, 7, 6, 4, 3, 2, 1]
        nbr_contributing = [nbr[i] if ldd_nbr[i] == seq_contributing[i] else None for i in range(8)]
        nbr_contributing = [x for x in nbr_contributing if x is not None]
        return nbr_contributing

    loc_to_visit = [(lon_snapped, lat_snapped)]
    loc_visited = []

    while len(loc_to_visit) > 0:
        loc = loc_to_visit.pop(0)
        loc_visited.append(loc)
        x, y = loc
        nbr = get_nbr_list(x, y)
        ldd_nbr = get_nbr_ldd(nbr)
        nbr_contributing = get_nbr_contributing(nbr, ldd_nbr)
        for nbr in nbr_contributing:
            if nbr not in loc_visited and nbr not in loc_to_visit:
                loc_to_visit.append(nbr)
        # print(len(loc_to_visit), len(loc_visited))
    
    # print(f"Number of contributing cells: {len(loc_visited)}")

    nodes_coords = pd.DataFrame(loc_visited, columns = ['lon', 'lat'])

    # fig, ax = plt.subplots(1,1,figsize = (10, 10))
    # ax.scatter(nodes_coords['lon'], nodes_coords['lat'], s = 1, c = 'blue', label = 'Contributing Cells')
    # ax.scatter(lon_snapped, lat_snapped, c = 'red', s = 10, label = 'Outlet Location')
    # ax.legend()
    # ax.set_aspect('equal')
    # ax.grid()
    # plt.show()

    def get_edge(loc, res = resolution):
        lon, lat = loc
        ldd_val = ldd_loc(lon, lat)
        if ldd_val == 1: # South-West
            edge_loc = (lon-res, lat-res)
        elif ldd_val == 2: # South
            edge_loc = (lon, lat-res)
        elif ldd_val == 3: # South-East
            edge_loc = (lon+res, lat-res)
        elif ldd_val == 4: # West
            edge_loc = (lon-res, lat)
        elif ldd_val == 6: # East
            edge_loc = (lon+res, lat)
        elif ldd_val == 7: # North-West
            edge_loc = (lon-res, lat+res)
        elif ldd_val == 8: # North
            edge_loc = (lon, lat+res)
        elif ldd_val == 9: # North-East
            edge_loc = (lon+res, lat+res)
        elif ldd_val == 5: # Pit
            edge_loc = None
        if edge_loc is not None:
            edge_loc = (round(edge_loc[0], 3), round(edge_loc[1], 3))
        return edge_loc

    edges = [(loc, get_edge(loc)) for loc in loc_visited[::-1][:-1]]
    # print(len(edges))

    # edges: ((lon_1, lat_1), (lon_2, lat_2)) -> edge_list: (idx_1, idx_2)
    def replace_loc_by_idx(loc, nodes_coords):
        return nodes_coords[(nodes_coords['lon'] == loc[0]) & (nodes_coords['lat'] == loc[1])].index[0]
    
    edges_idx = [(replace_loc_by_idx(edge[0], nodes_coords), replace_loc_by_idx(edge[1], nodes_coords)) for edge in edges if edge[1] is not None]

    
    # os.makedirs(os.path.join(SAVE_PATH, 'graph_files'), exist_ok = True)
    # os.makedirs(os.path.join(SAVE_PATH, 'graph_files', huc), exist_ok = True)
    # os.makedirs(os.path.join(SAVE_PATH, 'graph_files', huc, gauge_id), exist_ok = True)
    # SAVEPATH = os.path.join(SAVE_PATH, 'graph_files', huc, gauge_id)

    # nodes_coords.to_csv(os.path.join(SAVEPATH, 'nodes_coords.csv'), index = True)

    edges = pd.DataFrame(edges, columns = ['from', 'to'])
    edges_idx = pd.DataFrame(edges_idx, columns = ['from', 'to'])
    edges['from_idx'] = edges_idx['from']
    edges['to_idx'] = edges_idx['to']
    # edges.to_csv(os.path.join(SAVEPATH, 'edges.csv'), index = True)

    # # Return number of nodes and edges
    # num_nodes = len(nodes_coords)
    # num_edges = len(edges)
    # return num_nodes, num_edges

    return nodes_coords, edges

In [12]:
def snap_on_grid(
    grid_uparea: xr.DataArray,
    gauge_lon: float,
    gauge_lat: float,
    gauge_uparea: float,
    resolution: float,
    grid_ldd: xr.DataArray,
    huc: str,
    gauge_id: str
    ):
    assert 'lat' in grid_uparea.coords, 'latitude not in grid_uparea.coords'
    assert 'lon' in grid_uparea.coords, 'longitude not in grid_uparea.coords'
    is_lat_increasing = grid_uparea.lat[1].values > grid_uparea.lat[0].values
    is_lon_increasing = grid_uparea.lon[1].values > grid_uparea.lon[0].values
    if not is_lat_increasing:
        grid_uparea = grid_uparea.sortby('lat')
    if not is_lon_increasing:
        grid_uparea = grid_uparea.sortby('lon')

    # Ensure that both lat and lon are round(x, 3)
    grid_uparea['lat'] = grid_uparea['lat'].round(3)
    grid_uparea['lon'] = grid_uparea['lon'].round(3)

    grid_ldd['lat'] = grid_ldd['lat'].round(3)
    grid_ldd['lon'] = grid_ldd['lon'].round(3)
    
    # Find nearest grid cell
    grid_nearest = grid_uparea.sel(
        lat = gauge_lat,
        lon = gauge_lon,
        method = 'nearest'
    )
    nearest_lat = float(grid_nearest.lat.values)
    nearest_lon = float(grid_nearest.lon.values)

    grid_uparea = grid_uparea.sel(
        lat = slice(nearest_lat - 2*resolution, nearest_lat + 2*resolution),
        lon = slice(nearest_lon - 2*resolution, nearest_lon + 2*resolution)
    )
    
    # fig, ax = plt.subplots()
    # grid_uparea.plot(ax = ax, cmap = 'Blues', edgecolor = 'k')
    # ax.plot(gauge_lon, gauge_lat, 'ro')
    # ax.plot(nearest_lon, nearest_lat, 'bo')
    # plt.show()

    # Create DataArray UPA (upstream area accordance) = cell_uparea/gauge_uparea if gauge_uparea > cell_uparea else gauge_uparea/cell_uparea
    UPA = grid_uparea.copy()
    UPA = xr.where(UPA > gauge_uparea, gauge_uparea / UPA, UPA / gauge_uparea)
    
    subbasins_gauge = subbasins[subbasins['gauge_id'] == gauge_id].reset_index(drop = True)

    def calculate_iou(lon_under_consideration, lat_under_consideration):
        nodes_coords, edges = get_contributing_region(grid_ldd, lon_under_consideration, lat_under_consideration)
        edge_list = edges.loc[:,['from_idx', 'to_idx']].values.tolist()

        # fig, ax = plt.subplots(1,1,figsize = (10, 10))
        # camels.plot(ax = ax, color = 'none', edgecolor = 'black', lw = 0.5)
        # for idx, row in nodes_coords.iterrows():
        #     x, y = row[['lon', 'lat']]
        #     ax.plot(x, y, 'o', color = 'gray', markersize = 2)
        #     ax.add_patch(plt.Rectangle((x-resolution/2, y-resolution/2), resolution, resolution, fill=None, edgecolor='cornflowerblue', lw=0.5))
        # ax.plot(nearest_lon, nearest_lat, 'o', color = 'red', markersize = 4)
        # ax.plot(gauge_lon, gauge_lat, '^', color = 'red', markersize = 4)
        # for edge in edge_list:
        #     from_idx, to_idx = edge
        #     from_x, from_y = nodes_coords.loc[from_idx, ['lon', 'lat']]
        #     to_x, to_y = nodes_coords.loc[to_idx, ['lon', 'lat']]
        #     ax.annotate("", xy=(to_x, to_y), xytext=(from_x, from_y), arrowprops=dict(arrowstyle="-|>", mutation_scale=7, color = 'cornflowerblue', lw = 0.5))
        # ax.set_aspect('equal')
        # plt.show()

        # IOU = area of intersection / area of union
        # Create low resolution polygon from contributing nodes
        contributing_cells = []
        for idx, row in nodes_coords.iterrows():
            x, y = row[['lon', 'lat']]
            cell = Polygon([
                (x - resolution/2, y - resolution/2),
                (x - resolution/2, y + resolution/2),
                (x + resolution/2, y + resolution/2),
                (x + resolution/2, y - resolution/2)
            ])
            contributing_cells.append(cell)
        low_res_polygon = unary_union(contributing_cells)

        # Get high-resolution polygon from CAMELS
        high_res_polygon = subbasins_gauge.geometry.iloc[0]

        # Calculate intersection and union areas
        intersection_area = low_res_polygon.intersection(high_res_polygon).area
        union_area = low_res_polygon.union(high_res_polygon).area

        # Calculate IoU
        iou = intersection_area / union_area

        # Update the plot to include the low_res_polygon and high_res_polygon
        # fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        # gpd.GeoSeries([high_res_polygon]).plot(ax=ax, color='none', edgecolor='black', lw=0.5)
        # gpd.GeoSeries([low_res_polygon]).plot(ax=ax, color='none', edgecolor='red', lw=1)
        # for idx, row in nodes_coords.iterrows():
        #     x, y = row[['lon', 'lat']]
        #     ax.plot(x, y, 'o', color='gray', markersize=2)
        # ax.plot(nearest_lon, nearest_lat, 'o', color='red', markersize=4)
        # ax.plot(gauge_lon, gauge_lat, '^', color='red', markersize=4)
        # for edge in edge_list:
        #     from_idx, to_idx = edge
        #     from_x, from_y = nodes_coords.loc[from_idx, ['lon', 'lat']]
        #     to_x, to_y = nodes_coords.loc[to_idx, ['lon', 'lat']]
        #     ax.annotate("", xy=(to_x, to_y), xytext=(from_x, from_y), arrowprops=dict(arrowstyle="-|>", mutation_scale=7, color='cornflowerblue', lw=0.5))
        # ax.set_aspect('equal')
        # plt.title(f"IoU: {iou:.4f}")
        # plt.show()

        # print(f"Intersection area: {intersection_area:.4f}")
        # print(f"Union area: {union_area:.4f}")
        # print(f"IoU: {iou:.4f}")  

        return iou

    # Calculate IoU for all grid cells
    iou = UPA.copy()
    for lon_val in UPA.lon.values:
        for lat_val in UPA.lat.values:
            # Only calculate where grid_uparea is not nan
            if not np.isnan(grid_uparea.sel(lon = lon_val, lat = lat_val).values):
                iou_val = calculate_iou(lon_val, lat_val)
                iou.loc[dict(lon = lon_val, lat = lat_val)] = iou_val
            else:
                iou.loc[dict(lon = lon_val, lat = lat_val)] = np.nan
            # iou.loc[dict(lon = lon_val, lat = lat_val)] = calculate_iou(lon_val, lat_val)  

    # ED (Euclidian distance) = \sqrt{ (1 - UPA)^2 + (1 - iou)^2 }
    ED = np.sqrt((1 - UPA)**2 + (1 - iou)**2)
    # Plot ED
    # fig, ax = plt.subplots()
    # ED.plot(ax = ax, cmap = 'Blues', edgecolor = 'k')
    # ax.plot(nearest_lon, nearest_lat, 'ro')
    # ax.plot(gauge_lon, gauge_lat, 'r^')
    # plt.show()

    # Dataframe of ED with columns lon, lat and ED
    ED_df = ED.to_dataframe().reset_index()
    ED_df = ED_df.rename(columns = {ds_varname: 'ED'})
    ED_df['huc'] = huc
    ED_df['gauge_id'] = gauge_id
    # # Distance to gauge location
    ED_df['distance'] = np.sqrt((ED_df['lon'] - gauge_lon)**2 + (ED_df['lat'] - gauge_lat)**2)
    # Sort by ED and then by distance
    ED_df = ED_df.sort_values(by = ['ED', 'distance'], ascending = [True, True])

    # Select first cell as snapped location.
    lon_snapped = ED_df['lon'].iloc[0]
    lat_snapped = ED_df['lat'].iloc[0]
    # Get nodes and edges
    nodes_coords, edges = get_contributing_region(grid_ldd, lon_snapped, lat_snapped)
    # Save nodes and edges
    SAVEPATH = os.path.join(SAVE_PATH, 'graph_files', huc, gauge_id)
    os.makedirs(SAVEPATH, exist_ok = True)
    nodes_coords.to_csv(os.path.join(SAVEPATH, 'nodes_coords.csv'), index = True)
    edges.to_csv(os.path.join(SAVEPATH, 'edges.csv'), index = True)
    # uparea at snapped_location
    uparea_snapped = grid_uparea.sel(lon = lon_snapped, lat = lat_snapped).values
    # iou at snapped_location
    iou_snapped = calculate_iou(lon_snapped, lat_snapped)
    # area_percent_difference
    area_percent_difference = (abs(gauge_uparea - uparea_snapped) / gauge_uparea) * 100

    num_nodes = len(nodes_coords)
    num_edges = len(edges)
    edge_list = edges.loc[:,['from_idx', 'to_idx']].values.tolist()

    fig, ax = plt.subplots(1,1,figsize = (15, 15))
    subbasins_gauge.plot(ax = ax, color = 'none', edgecolor = 'black', lw = 0.5)
    for idx, row in nodes_coords.iterrows():
        x, y = row[['lon', 'lat']]
        ax.plot(x, y, 'o', color = 'gray', markersize = 2)
        ax.add_patch(plt.Rectangle((x-resolution/2, y-resolution/2), resolution, resolution, fill=None, edgecolor='cornflowerblue', lw=0.5))
    ax.plot(nearest_lon, nearest_lat, 'o', color = 'red', markersize = 4)
    ax.plot(gauge_lon, gauge_lat, '^', color = 'red', markersize = 4)
    ax.plot(lon_snapped, lat_snapped, 'o', color = 'green', markersize = 4)
    for edge in edge_list:
        from_idx, to_idx = edge
        from_x, from_y = nodes_coords.loc[from_idx, ['lon', 'lat']]
        to_x, to_y = nodes_coords.loc[to_idx, ['lon', 'lat']]
        ax.annotate("", xy=(to_x, to_y), xytext=(from_x, from_y), arrowprops=dict(arrowstyle="-|>", mutation_scale=7, color = 'cornflowerblue', lw = 0.5))
    ax.set_aspect('equal')
    # plt.show()
    os.makedirs(os.path.join(SAVE_PATH, 'graph_plots', huc), exist_ok = True)
    plt.savefig(os.path.join(SAVE_PATH, 'graph_plots', huc, f"{gauge_id}.pdf"))
    plt.close()

    return lon_snapped, lat_snapped, uparea_snapped, iou_snapped, area_percent_difference, num_nodes, num_edges

# row = camels_attributes.iloc[0]
# temp = snap_on_grid(uparea, row['gauge_lon'], row['gauge_lat'], row['area_geospa_fabric'], resolution, ldd, row['huc_02'], row.name)

In [31]:
row = indiawris_basin.iloc[6]
temp = snap_on_grid(
    grid_uparea = uparea, 
    gauge_lon = row['MERIT_Longitude'],
    gauge_lat = row['MERIT_Latitude'],
    gauge_uparea = row['uparea (in km2)'],
    resolution = resolution, 
    grid_ldd = ldd, 
    huc = basin_name, 
    gauge_id = row['River Point Name']
    )

In [13]:
indiawris_basin_snapped = indiawris_basin.copy()
for idx, row in tqdm.tqdm(indiawris_basin_snapped.iterrows(), total = len(indiawris_basin_snapped)):
    try:
        lon_snapped, lat_snapped, uparea_snapped, iou_snapped, area_percent_difference, num_nodes, num_edges = snap_on_grid(
            grid_uparea = uparea, 
            gauge_lon = row['MERIT_Longitude'],
            gauge_lat = row['MERIT_Latitude'],
            gauge_uparea = row['uparea (in km2)'],
            resolution = resolution, 
            grid_ldd = ldd, 
            huc = basin_name, 
            gauge_id = row['River Point Name']
            )
    except:
        lon_snapped, lat_snapped, uparea_snapped, iou_snapped, area_percent_difference, num_nodes, num_edges = None, None, None, None, None, None, None
    indiawris_basin_snapped.loc[idx, 'snapped_lon'] = lon_snapped
    indiawris_basin_snapped.loc[idx, 'snapped_lat'] = lat_snapped
    indiawris_basin_snapped.loc[idx, 'snapped_uparea'] = uparea_snapped
    indiawris_basin_snapped.loc[idx, 'snapped_iou'] = iou_snapped
    indiawris_basin_snapped.loc[idx, 'area_percent_difference'] = area_percent_difference
    indiawris_basin_snapped.loc[idx, 'num_nodes'] = num_nodes
    indiawris_basin_snapped.loc[idx, 'num_edges'] = num_edges
indiawris_basin_snapped.head()

100%|██████████| 18/18 [00:17<00:00,  1.02it/s]


,River Point Name,Latitude,Longitude,NaNs,Days with Observations,Zero of Gauge,MERIT_Longitude,MERIT_Latitude,Gauge_MERIT_Distance,uparea (in m2),uparea (in km2),snapped_lon,snapped_lat,snapped_uparea,snapped_iou,area_percent_difference,num_nodes,num_edges
0,andhiyar_khore,21.833889,81.597500,5.30,14184,252.0,81.597500,21.834167,30.887,2.166368e+09,2166.368402,81.25,22.25,2853.602539,0.512368,31.722864,1.0,0.0
1,bamnidhi,21.908486,82.713640,4.45,14311,223.0,82.710000,21.910000,411.522,9.811223e+09,9811.223484,82.75,22.25,11373.951172,0.639430,15.927957,4.0,3.0
2,baronda,20.910000,81.888056,6.64,13983,283.0,81.888333,20.910000,28.853,3.193768e+09,3193.768107,82.25,20.25,2891.693359,0.362836,9.458254,1.0,0.0
3,basantpur,21.738500,82.785942,4.64,14282,206.0,82.778333,21.722500,1944.963,5.898489e+10,58984.885999,82.75,21.75,65959.007812,0.798565,11.823573,23.0,22.0
4,ghatora,22.048592,82.221956,5.35,14176,246.0,82.222500,22.047500,133.739,3.037212e+09,3037.211720,82.25,22.25,2853.602539,0.377117,6.045319,1.0,0.0


In [14]:
indiawris_basin_snapped.to_csv(os.path.join(SAVE_PATH, f'graph_attributes_{basin_name}.csv'), index = True)